In [1]:
import numpy as np
from scipy.stats import truncnorm
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
### definimos una clase jugador
## vamos a suponer que cada jugador conoce un numero al azar de jugadores entre 1 y 200
## (se podría mejorar armandolo con un grafo)
## vamos a suponer que cada jugador tiene un parámetro de desconfianza
class jugador():
    def __init__(self,dinero,min_conocidos = 1,max_conocidos = 200):
        self.dinero = dinero
        self.numero_de_conocidos = np.random.randint(1,max_conocidos+1) 
        mu = 0.5 ## media de la desconfianza
        sig = 0.3 ## desvío de las desconfianza
        self.desconfianza = 1 - truncnorm.rvs((0 - mu) / sig, (1 - mu) / sig, loc=mu, scale=sig)
    def get_estado(self):
        return(self.dinero)
    def get_numero_conocidos(self):
        return(self.numero_de_conocidos)
    def get_desconfianza(self):
        return self.desconfianza
    def transferir(self,otro_jugador,monto):
        self.dinero -= monto
        otro_jugador.dinero += monto

Por simplicidad vamos a suponer que hay N jugadores que empiezan con 1000 pesos cada uno. Si un jugador se queda sin dinero, no puede seguir jugando. 

In [3]:
dinero_inicial = 100 #con cuanta plata arranca cada jugador
N = 1000 #Numero de jugadores
selectores_iniciales = 1 ## numero de jugadores que comienzan con el juego.
tamanio_grupo= 8 ## numero de jugadores necesarios para que se mueva plata
transferencia = 10 ## plata que transfieren

pool_jugadores = [jugador(dinero_inicial) for i in range(N)] ## pool inicial de jugadores
iniciadores = list(np.random.choice(pool_jugadores,selectores_iniciales))
salidos = []
nuevos_iniciadores = []

In [4]:
np.random.seed(123)

In [5]:
for PRUEBA in range(100):
    for iniciador in iniciadores:
         ## solo toma no iniciadores con plata
        posibles_victimas_iniciador = [x for x in pool_jugadores if x not in iniciadores and x.get_estado() >= 10]
        if len(posibles_victimas_iniciador) <= 0:
            salidos.append(iniciador) ## retiro el iniciador si no consigue victimas
            continue
        else:
            posibles_victimas_iniciador_seleccionadas = np.random.choice(posibles_victimas_iniciador,min(len(posibles_victimas_iniciador),iniciador.get_numero_conocidos()),replace=False)
    ## ahora, para seleccionar. 
    ## Supongamos que la aceptación o no viene de una bernoulli con probabilidad = 1 - desconfianza
    ## esto es general pero se podría hacer con un grafo con las aristas pesadas por confianza
    ## vamos a pasar por todos los conocidos. Si llega a 8 se corta
    ## si no llega a 8, se frustra y no juega mas

        victimas = []
        for victima in posibles_victimas_iniciador_seleccionadas:
            if np.random.binomial(1,victima.get_desconfianza()):
                victimas.append(victima)
            if len(victimas) == tamanio_grupo:
                break

        ## ahora, si completó el grupo, cada uno le pasa $10 al jugador. 
        ## Luego se convierte en iniciador
        if len(victimas) == tamanio_grupo:
            for victima in victimas:
                victima.transferir(iniciador,transferencia)
                ## si le queda dinero, la agregamos a la lista. Si no, pierde y se va
                if victima.get_estado() > 0:
                    nuevos_iniciadores.append(victima)
                else: ## si no le queda dinero, sale del juego
                    salidos.append(victima)
        else: 
            salidos.append(iniciador)
            continue

    iniciadores += nuevos_iniciadores
    iniciadores = list(set(iniciadores))
    for salido in salidos:
        if salido in iniciadores:
            iniciadores.remove(salido)
        if salido in pool_jugadores:
            pool_jugadores.remove(salido)
    
    print("jugadores que son iniciadores: ",len(iniciadores))
    print("jugadores que aun tienen algo de dinero: ", len(pool_jugadores))
    if (len(iniciadores) == 0) | (len(pool_jugadores) == 0):
        print("MURIO")
        break

jugadores que son iniciadores:  9
jugadores que aun tienen algo de dinero:  1000
jugadores que son iniciadores:  77
jugadores que aun tienen algo de dinero:  1000
jugadores que son iniciadores:  491
jugadores que aun tienen algo de dinero:  997
jugadores que son iniciadores:  730
jugadores que aun tienen algo de dinero:  745
jugadores que son iniciadores:  0
jugadores que aun tienen algo de dinero:  15
MURIO


In [6]:
## veamos los que jugaron
plata_afuera = [x.get_estado() for x in salidos]
grupos = pd.cut(plata_afuera,[0,1,100,np.inf],right=False).value_counts()
grupos / grupos.sum()*100

[0.0, 1.0)      19.898477
[1.0, 100.0)    35.634518
[100.0, inf)    44.467005
dtype: float64